#2.2.1

In [2]:
import numpy as np
import pandas as pd

data = pd.read_excel('Classification iris.xlsx')
train_data = pd.DataFrame()
test_data = pd.DataFrame()

#Extract the first 70% index and concatenate to the data frame.
for category in data['class'].unique():
    category_data = data[data['class'] == category]
    split_index = int(len(category_data) * 0.7)
    train_data = pd.concat([train_data, category_data.iloc[:split_index]])
    test_data = pd.concat([test_data, category_data.iloc[split_index:]])

# extract the first column
training_set_ids = [insid for insid in train_data.iloc[:, 0]]
testing_set_ids = [insid for insid in test_data.iloc[:, 0]]
    
print("Q2.2.1 Split training set and test set:")
print("\nTraining set", training_set_ids)
print("\nTest set", testing_set_ids)


Q2.2.1 Split training set and test set:

Training set [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135]

Test set [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]


#2.2.2

In [4]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

#---------------------------------------------data processing--------------------------------------------
data = pd.read_excel('Classification iris.xlsx')

train_data = pd.DataFrame()
test_data = pd.DataFrame()

#Extract the first 70% index and concatenate to the data frame.
for category in data['class'].unique():
    category_data = data[data['class'] == category]
    split_index = int(len(category_data) * 0.7)
    train_data = pd.concat([train_data, category_data.iloc[:split_index]])
    test_data = pd.concat([test_data, category_data.iloc[split_index:]])

# extract X_train and y_train
X_train = train_data.drop(columns=['instance_id', 'class'])
y_train = train_data['class']

# extract X_test and y_test
X_test = test_data.drop(columns=['instance_id', 'class'])
y_test = test_data['class']


# Map class names into 0,1,2
class_names = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
y_train = y_train.map(class_names)
y_test = y_test.map(class_names)

#---------------------------------------------Training svm--------------------------------------------

svm_model = SVC(kernel='linear', C=1e5, decision_function_shape='ovr')
svm_model.fit(X_train, y_train)

#Predict values
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)

#Calculate train error and test error
train_error = 1 - accuracy_score(y_train, y_train_pred)
test_error = 1 - accuracy_score(y_test, y_test_pred)

print("Q2.2.2 Calculation using Standard SVM Model:")
print("total training error:", train_error, "total testing error:", test_error)

#Extract w and b
w = svm_model.coef_
b = svm_model.intercept_

# support vector indices and numebrs
support_vector_indices = svm_model.support_
support_vector_nums = svm_model.n_support_

# create a list to store linear separable class
linear_separable_classes = []

for class_name, class_label in class_names.items():
    # extract train data and test data for each class
    class_train_data = train_data[train_data['class'] == class_name]
    class_test_data = test_data[test_data['class'] == class_name]

    # predict target y for train data and test data
    class_y_train_pred = svm_model.predict(class_train_data[['sepal length', 'sepal width', 'petal length', 'petal width']])
    class_y_test_pred = svm_model.predict(class_test_data[['sepal length', 'sepal width', 'petal length', 'petal width']])

    # calculate train error and test error
    class_train_error = 1 - accuracy_score(class_train_data['class'].map(class_names), class_y_train_pred)
    class_test_error = 1 - accuracy_score(class_test_data['class'].map(class_names), class_y_test_pred)

    # support vector indices for different class
    if class_label == 0:
        class_support_vector_indices = support_vector_indices[:support_vector_nums[0]]
    elif class_label == 1:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0]:support_vector_nums[0] + support_vector_nums[1]]
    elif class_label == 2:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0] + support_vector_nums[1]:]
    
    print(f"\nclass {class_name}:")
    print("training error:", class_train_error, ", testing error:", class_test_error)
    print("w:", w[class_label])
    print("b:", b[class_label])
    print("support vector indices:", class_support_vector_indices)

    # judge whether linear separable by accuracy
    if class_train_error < 0.05 and class_test_error < 0.05:
        linear_separable_classes.append(class_name)

print("Linear separable classes:", linear_separable_classes)


Q2.2.2 Calculation using Standard SVM Model:
total training error: 0.01904761904761909 total testing error: 0.0

class Iris-setosa:
training error: 0.0 , testing error: 0.0
w: [ 0.00945021  0.53765319 -0.82682922 -0.38216595]
b: 0.774099177589861
support vector indices: [23 24]

class Iris-versicolor:
training error: 0.02857142857142858 , testing error: 0.0
w: [-0.00708258  0.17885072 -0.53832399 -0.29218158]
b: 1.5070188241897433
support vector indices: [42 55 57 62 68]

class Iris-virginica:
training error: 0.02857142857142858 , testing error: 0.0
w: [  3.64650338   5.176364    -7.42852538 -11.00241583]
b: 17.570392115365593
support vector indices: [ 76  96  97  99 103]
Linear separable classes: ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']


#2.2.3

In [6]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np

#---------------------------------------------data processing--------------------------------------------
data = pd.read_excel('Classification iris.xlsx')

train_data = pd.DataFrame()
test_data = pd.DataFrame()

#Extract the first 70% index and concatenate to the data frame.
for category in data['class'].unique():
    category_data = data[data['class'] == category]
    split_index = int(len(category_data) * 0.7)
    train_data = pd.concat([train_data, category_data.iloc[:split_index]])
    test_data = pd.concat([test_data, category_data.iloc[split_index:]])

# extract X_train and y_train
X_train = train_data.drop(columns=['instance_id', 'class'])
y_train = train_data['class']

# extract X_test and y_test
X_test = test_data.drop(columns=['instance_id', 'class'])
y_test = test_data['class']

# Map class names into 0,1,2
class_names = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
y_train = y_train.map(class_names)
y_test = y_test.map(class_names)

print("Q2.2.3 Calculation using SVM with Slack Variables (C = 0.25 × t, where t = 1, . . . , 4):")

#---------------------------------------------Training svm--------------------------------------------
for t in range(1,5):
    print("--------------------------------------------------")
    print("C=", 0.25*t)

    # train svm
    svm_model = SVC(kernel='linear', C=0.25*t, decision_function_shape='ovr')
    svm_model.fit(X_train, y_train)

    # predict values
    y_train_pred = svm_model.predict(X_train)
    y_test_pred = svm_model.predict(X_test)

    # train error and test error
    train_error = 1 - accuracy_score(y_train, y_train_pred)
    test_error = 1 - accuracy_score(y_test, y_test_pred)
    
    print("total training error:", train_error, "total testing error:", test_error)

    # w and b
    w = svm_model.coef_
    b = svm_model.intercept_

    # support vectors
    support_vector_indices = svm_model.support_
    support_vector_nums = svm_model.n_support_
        
    for class_name, class_label in class_names.items():
        # extract class train data and test data
        class_train_data = train_data[train_data['class'] == class_name]
        class_test_data = test_data[test_data['class'] == class_name]

        # predict values
        class_y_train_pred = svm_model.predict(class_train_data[['sepal length', 'sepal width', 'petal length', 'petal width']])
        class_y_test_pred = svm_model.predict(class_test_data[['sepal length', 'sepal width', 'petal length', 'petal width']])

        # train error and test error for class
        class_train_error = 1 - accuracy_score(class_train_data['class'].map(class_names), class_y_train_pred)
        class_test_error = 1 - accuracy_score(class_test_data['class'].map(class_names), class_y_test_pred)

        # support vector indices
        if class_label == 0:
            class_support_vector_indices = support_vector_indices[:support_vector_nums[0]]
        elif class_label == 1:
            class_support_vector_indices = support_vector_indices[support_vector_nums[0]:support_vector_nums[0] + support_vector_nums[1]]
        elif class_label == 2:
            class_support_vector_indices = support_vector_indices[support_vector_nums[0] + support_vector_nums[1]:]

        # since y_train consists of 0,1,2, we need to convert it to binary array, then calculate slack variable.
        support_vectors = X_train.iloc[class_support_vector_indices]
        binary_y_train = np.where(y_train == class_label, 1, -1)
        slack_variables = 1 - binary_y_train[class_support_vector_indices] * (np.dot(support_vectors, w[class_label]) + b[class_label])
        
        print(f"\nclass {class_name}:")
        print("training error:", class_train_error, ", testing error:", class_test_error)
        print("w:", w[class_label])
        print("b:", b[class_label])
        print("support vector indices:", class_support_vector_indices)
        print("slack variable:", slack_variables)
        

Q2.2.3 Calculation using SVM with Slack Variables (C = 0.25 × t, where t = 1, . . . , 4):
--------------------------------------------------
C= 0.25
total training error: 0.04761904761904767 total testing error: 0.022222222222222254

class Iris-setosa:
training error: 0.0 , testing error: 0.0
w: [-0.16774186  0.41854832 -0.78709665 -0.31854832]
b: 1.9412904435714369
support vector indices: [23 24]
slack variable: [ 3.03220571e-02 -5.02063441e-07]

class Iris-versicolor:
training error: 0.02857142857142858 , testing error: 0.0
w: [-0.00708258  0.17885072 -0.53832399 -0.29218158]
b: 1.5070188241897433
support vector indices: [35 37 39 40 41 42 48 51 53 55 57 58 61 62 63 64 68 69]
slack variable: [1.9094139  2.06347367 1.95279866 1.83486388 1.94500734 1.16709484
 1.9566948  1.85682179 2.00415191 2.0723281  2.16653455 1.91614355
 2.03337007 2.19221094 1.8775399  1.24465556 2.26552259 1.85540528]

class Iris-virginica:
training error: 0.11428571428571432 , testing error: 0.06666666666666665

#2.2.4

In [8]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler

#---------------------------------------------data processing--------------------------------------------
data = pd.read_excel('Classification iris.xlsx')

train_data = pd.DataFrame()
test_data = pd.DataFrame()

#Extract the first 70% index and concatenate to the data frame.
for category in data['class'].unique():
    category_data = data[data['class'] == category]
    split_index = int(len(category_data) * 0.7)
    train_data = pd.concat([train_data, category_data.iloc[:split_index]])
    test_data = pd.concat([test_data, category_data.iloc[split_index:]])

# extract X_train and y_train
X_train = train_data.drop(columns=['instance_id', 'class'])
y_train = train_data['class']

# extract X_test and y_test
X_test = test_data.drop(columns=['instance_id', 'class'])
y_test = test_data['class']

# Map class names into 0,1,2
class_names = {"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2}
y_train = y_train.map(class_names)
y_test = y_test.map(class_names)

print("Q2.2.4 Calculation using SVM with Kernel Functions:")

#---------------------------------------------Training svm--------------------------------------------    


#---------------------------2nd-poly--------------------------------

print("------------------------------------------")
print("(a) 2nd-order Polynomial Kernel,")
svm_model_p2 = SVC(kernel='poly', degree=2, C=1e5, decision_function_shape='ovr')
svm_model_p2.fit(X_train, y_train)

y_train_pred = svm_model_p2.predict(X_train)
y_test_pred = svm_model_p2.predict(X_test)

train_error = 1 - accuracy_score(y_train, y_train_pred)
test_error = 1 - accuracy_score(y_test, y_test_pred)

print("total training error:", train_error, "total testing error:", test_error)

# w = svm_model.coef_
# b = svm_model.intercept_

# extratc support vector indices and numbers
support_vector_indices = svm_model_p2.support_
support_vector_nums = svm_model_p2.n_support_


for class_name, class_label in class_names.items():
    # extract train data and test data for each class
    class_train_data = train_data[train_data['class'] == class_name]
    class_test_data = test_data[test_data['class'] == class_name]

    # predict target y for train data and test data
    class_y_train_pred = svm_model_p2.predict(class_train_data[['sepal length', 'sepal width', 'petal length', 'petal width']])
    class_y_test_pred = svm_model_p2.predict(class_test_data[['sepal length', 'sepal width', 'petal length', 'petal width']])

    # calculate train error and test error
    class_train_error = 1 - accuracy_score(class_train_data['class'].map(class_names), class_y_train_pred)
    class_test_error = 1 - accuracy_score(class_test_data['class'].map(class_names), class_y_test_pred)

    # extract support vecctor indices for each class based on class label
    if class_label == 0:
        class_support_vector_indices = support_vector_indices[:support_vector_nums[0]]
    elif class_label == 1:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0]:support_vector_nums[0] + support_vector_nums[1]]
    elif class_label == 2:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0] + support_vector_nums[1]:]
    
    
    print(f"\nclass {class_name}:")
    print("training error:", class_train_error, ", testing error:", class_test_error)
    print("w: ")
    print("b: ")
    print("support vector indices:", class_support_vector_indices)

#---------------------------3nd-poly--------------------------------

print("------------------------------------------")
print("(b) 3nd-order Polynomial Kernel,")

svm_model_p3 = SVC(kernel='poly', degree=3, C=1e5, decision_function_shape='ovr')
svm_model_p3.fit(X_train, y_train)

y_train_pred = svm_model_p3.predict(X_train)
y_test_pred = svm_model_p3.predict(X_test)

train_error = 1 - accuracy_score(y_train, y_train_pred)
test_error = 1 - accuracy_score(y_test, y_test_pred)

print("total training error:", train_error, "total testing error:", test_error)

# w = svm_model.coef_
# b = svm_model.intercept_

# extratc support vector indices and numbers
support_vector_indices = svm_model_p3.support_
support_vector_nums = svm_model_p3.n_support_

for class_name, class_label in class_names.items():
    # extract train data and test data for each class
    class_train_data = train_data[train_data['class'] == class_name]
    class_test_data = test_data[test_data['class'] == class_name]

    # predict target y for train data and test data
    class_y_train_pred = svm_model_p3.predict(class_train_data[['sepal length', 'sepal width', 'petal length', 'petal width']])
    class_y_test_pred = svm_model_p3.predict(class_test_data[['sepal length', 'sepal width', 'petal length', 'petal width']])

    # calculate train error and test error
    class_train_error = 1 - accuracy_score(class_train_data['class'].map(class_names), class_y_train_pred)
    class_test_error = 1 - accuracy_score(class_test_data['class'].map(class_names), class_y_test_pred)

    # extract support vecctor indices for each class based on class label
    if class_label == 0:
        class_support_vector_indices = support_vector_indices[:support_vector_nums[0]]
    elif class_label == 1:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0]:support_vector_nums[0] + support_vector_nums[1]]
    elif class_label == 2:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0] + support_vector_nums[1]:]
    
    
    print(f"\nclass {class_name}:")
    print("training error:", class_train_error, ", testing error:", class_test_error)
    print("w: ")
    print("b: ")
    print("support vector indices:", class_support_vector_indices)


#---------------------------rbf--------------------------------

print("------------------------------------------")
print("(c) Radial Basis Function Kernel with σ = 1,")

svm_model_rbf = SVC(kernel='rbf', gamma=0.5, C=1e5, decision_function_shape='ovr')
svm_model_rbf.fit(X_train, y_train)

y_train_pred = svm_model_rbf.predict(X_train)
y_test_pred = svm_model_rbf.predict(X_test)

train_error = 1 - accuracy_score(y_train, y_train_pred)
test_error = 1 - accuracy_score(y_test, y_test_pred)

print("total training error:", train_error, "total testing error:", test_error)

# w = svm_model.coef_
# b = svm_model.intercept_

# extratc support vector indices and numbers
support_vector_indices = svm_model_rbf.support_
support_vector_nums = svm_model_rbf.n_support_


for class_name, class_label in class_names.items():
    # extract train data and test data for each class
    class_train_data = train_data[train_data['class'] == class_name]
    class_test_data = test_data[test_data['class'] == class_name]

    # predict target y for train data and test data
    class_y_train_pred = svm_model_rbf.predict(class_train_data[['sepal length', 'sepal width', 'petal length', 'petal width']])
    class_y_test_pred = svm_model_rbf.predict(class_test_data[['sepal length', 'sepal width', 'petal length', 'petal width']])

    # calculate train error and test error
    class_train_error = 1 - accuracy_score(class_train_data['class'].map(class_names), class_y_train_pred)
    class_test_error = 1 - accuracy_score(class_test_data['class'].map(class_names), class_y_test_pred)

    # extract support vecctor indices for each class based on class label
    if class_label == 0:
        class_support_vector_indices = support_vector_indices[:support_vector_nums[0]]
    elif class_label == 1:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0]:support_vector_nums[0] + support_vector_nums[1]]
    elif class_label == 2:
        class_support_vector_indices = support_vector_indices[support_vector_nums[0] + support_vector_nums[1]:]
    
    
    print(f"\nclass {class_name}:")
    print("training error:", class_train_error, ", testing error:", class_test_error)
    print("w: ")
    print("b: ")
    print("support vector indices:", class_support_vector_indices)

#---------------------------sigmoid--------------------------------

print("------------------------------------------")
print("(d) Sigmoidal Kernel with σ = 1,")

# train data and test data
X_train = train_data.drop(columns=['instance_id', 'class'])
y_train = train_data['class']
X_test = test_data.drop(columns=['instance_id', 'class'])
y_test = test_data['class']

# feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# train svm
svm_model = SVC(kernel='sigmoid', C=1e5, gamma=0.5, decision_function_shape='ovr')
svm_model.fit(X_train, y_train)

# claculate training error, test error
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)
train_error = 1 - accuracy_score(y_train, y_train_pred)
test_error = 1 - accuracy_score(y_test, y_test_pred)

print("total training error:", train_error, ", total testing error:", test_error)



support_vector_indices = svm_model.support_
support_vector_nums = svm_model.n_support_


for class_name in data['class'].unique():
    class_indices_train = (y_train == class_name)
    class_indices_test = (y_test == class_name)
    
    y_train_class_pred = y_train_pred[class_indices_train]
    y_test_class_pred = y_test_pred[class_indices_test]
    
    train_class_error = 1 - accuracy_score(y_train[class_indices_train], y_train_class_pred)
    test_class_error = 1 - accuracy_score(y_test[class_indices_test], y_test_class_pred)
    
    # support vector indices
    if class_name == "Iris-setosa":
        class_support_vector_indices = support_vector_indices[:support_vector_nums[0]]
    elif class_name == "Iris-versicolor":
        class_support_vector_indices = support_vector_indices[support_vector_nums[0]:support_vector_nums[0] + support_vector_nums[1]]
    elif class_name == "Iris-virginica":
        class_support_vector_indices = support_vector_indices[support_vector_nums[0] + support_vector_nums[1]:]

    # since y_train consists of 0,1,2, we need to convert it to binary array, then calculate slack variable.
    support_vectors = X_train[class_support_vector_indices]
    
    print(f"\nclass {class_name}:")
    print("training error:", train_class_error, ", testing error:", test_class_error)
    print("w:")
    print("b:")
    print("support vector indices:", class_support_vector_indices)

Q2.2.4 Calculation using SVM with Kernel Functions:
------------------------------------------
(a) 2nd-order Polynomial Kernel,
total training error: 0.00952380952380949 total testing error: 0.0

class Iris-setosa:
training error: 0.0 , testing error: 0.0
w: 
b: 
support vector indices: [23 24]

class Iris-versicolor:
training error: 0.02857142857142858 , testing error: 0.0
w: 
b: 
support vector indices: [42 55 57 58 62 68]

class Iris-virginica:
training error: 0.0 , testing error: 0.0
w: 
b: 
support vector indices: [ 76  89  96  97 103]
------------------------------------------
(b) 3nd-order Polynomial Kernel,
total training error: 0.0 total testing error: 0.0

class Iris-setosa:
training error: 0.0 , testing error: 0.0
w: 
b: 
support vector indices: [23 24]

class Iris-versicolor:
training error: 0.0 , testing error: 0.0
w: 
b: 
support vector indices: [42 55 57 58 68]

class Iris-virginica:
training error: 0.0 , testing error: 0.0
w: 
b: 
support vector indices: [ 76  89  93  9